In [ ]:
!nvidia-smi

Thu Nov  2 13:29:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset
data = load_dataset("re2panda/click_bate_1000_train_test")

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['input', 'output'],
        num_rows: 2400
    })
})

In [ ]:
sample_num = list(range(0, 2400, 24))

In [ ]:
len(sample_num)

100

In [ ]:
data['validation'] = data['validation'].select(sample_num)

In [ ]:
data['validation'].find("정상기사")

AttributeError: ignored

In [ ]:
data = data.shuffle(seed=42)

In [ ]:
data = data.map(
    lambda x: {'text': f"### 질문: {x['input']}\n\n### 답변: {x['output']}" }
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset

data = load_dataset("re2panda/click_bate_1000_train_test")
data["train"]

data = data.map(
    lambda x: {'text': f"### 질문: {x['input']}\n\n### 답변: {x['output']}" }
)

Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-1.3b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

OSError: ignored

In [ ]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

In [ ]:
data['train'][0]['text']

'### 질문: 기사제목: f김영록 예비후보 선거사무소 개소 “압도적 승리로 보답” 기사내용: f김영록 더불어민주당 전남도지사 예비후보가 9일 선거사무소 ‘대도약 행복캠프’를 개소하고 6·1지방선거에서의 압도적 승리를 위한 대장정에 본격 나섰다.\n순천시 팔마로 MJ빌딩 1층에 마련된 ‘대도약 행복캠프’에는 선거사무소 개소를 축하하기 위한 각계각층의 발길이 이어지며 대성황을 이뤘다.\n행사 시작 전부터 구름 인파가 몰리기 시작해 5000여 명의 지지자들이 선거사무소 개소식을 찾아 승리를 위한 힘찬 발걸음에 동참했다.\n이 자리에는 문희상 전 국회의장과 원혜영·우원식 전 원내대표, 허경만 전 전남도지사, 천정배 전 장관, 서동용 국회의원, 강기갑 전 국회의원 등이 참석해 개소를 축하했다.\n특히 정세균 전 국무총리와 윤호중 당 비상대책위원장, 박홍근 원내대표, 김상희 국회부의장, 송영길 서울시장 후보, 박남춘 인천시장 후보 등이 영상과 축전으로 ‘대도약 행복캠프’ 개소를 축하하며 김 후보의 압도적 승리에 힘을 보탰다.\n이밖에 지역 기업인과 지역 원로, 문화예술, 사회복지, 장애인, 종교, 언론인, 보호단체 등 각계각층 인사와 전남도민들이 대거 참석해 빛을 발했다.\n이날 개소식에 참석한 참석자들은 김 후보가 전남도지사로 전남도정을 이끌었던 민선 7기의 역사적 성과를 높이 평가하며 중단없는 전남발전의 필요성에 대해 한 목소리를 냈다.\n문 전 의장은 “지성과 열정·용기를 갖춘, 일 잘하는 김 후보가 전남발전을 위해 계속해야 한다”며 “호남의 민주정치를 이어나갈 최적의 후보다. 호남이 똘똘 뭉쳐 민주당의 저력을 보여주자”고 힘을 실었다.\n우 전 대표는 “김 후보는 전남의 행복시대를 확실하게 열 수 있는 사람이다. 공약이행률, 전남예산, 전남 총생산 등 각종 수치가 말해주고 있다. 전남이 새로운 시작, 새로운 비전을 준비하고 있다”며 “새로운 대한민국 건설을 위해 일 잘하고, 전남을 발전시킬 수 있는 김 후보에 힘을 실어달라”고 말했다.\n김 후보는 이날 ‘승리 

In [ ]:
data['train'][0]['text']

'### 질문: 기사제목: f김영록 예비후보 선거사무소 개소 “압도적 승리로 보답” 기사내용: f김영록 더불어민주당 전남도지사 예비후보가 9일 선거사무소 ‘대도약 행복캠프’를 개소하고 6·1지방선거에서의 압도적 승리를 위한 대장정에 본격 나섰다.\n순천시 팔마로 MJ빌딩 1층에 마련된 ‘대도약 행복캠프’에는 선거사무소 개소를 축하하기 위한 각계각층의 발길이 이어지며 대성황을 이뤘다.\n행사 시작 전부터 구름 인파가 몰리기 시작해 5000여 명의 지지자들이 선거사무소 개소식을 찾아 승리를 위한 힘찬 발걸음에 동참했다.\n이 자리에는 문희상 전 국회의장과 원혜영·우원식 전 원내대표, 허경만 전 전남도지사, 천정배 전 장관, 서동용 국회의원, 강기갑 전 국회의원 등이 참석해 개소를 축하했다.\n특히 정세균 전 국무총리와 윤호중 당 비상대책위원장, 박홍근 원내대표, 김상희 국회부의장, 송영길 서울시장 후보, 박남춘 인천시장 후보 등이 영상과 축전으로 ‘대도약 행복캠프’ 개소를 축하하며 김 후보의 압도적 승리에 힘을 보탰다.\n이밖에 지역 기업인과 지역 원로, 문화예술, 사회복지, 장애인, 종교, 언론인, 보호단체 등 각계각층 인사와 전남도민들이 대거 참석해 빛을 발했다.\n이날 개소식에 참석한 참석자들은 김 후보가 전남도지사로 전남도정을 이끌었던 민선 7기의 역사적 성과를 높이 평가하며 중단없는 전남발전의 필요성에 대해 한 목소리를 냈다.\n문 전 의장은 “지성과 열정·용기를 갖춘, 일 잘하는 김 후보가 전남발전을 위해 계속해야 한다”며 “호남의 민주정치를 이어나갈 최적의 후보다. 호남이 똘똘 뭉쳐 민주당의 저력을 보여주자”고 힘을 실었다.\n우 전 대표는 “김 후보는 전남의 행복시대를 확실하게 열 수 있는 사람이다. 공약이행률, 전남예산, 전남 총생산 등 각종 수치가 말해주고 있다. 전남이 새로운 시작, 새로운 비전을 준비하고 있다”며 “새로운 대한민국 건설을 위해 일 잘하고, 전남을 발전시킬 수 있는 김 후보에 힘을 실어달라”고 말했다.\n김 후보는 이날 ‘승리 

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1572864 || all params: 729403392 || trainable%: 0.21563705588032142


In [ ]:
# CUTOFF_LEN = 3930

In [ ]:
# def generate_prompt(data_point):
#     return f"""
# ### 질문:
# {data_point["input"]}
# ### 답변:
# {data_point["output"]}<|endoftext|> """


# def tokenize(prompt, add_eos_token=True):
#     result = tokenizer(
#         prompt,
#         truncation=True,
#         max_length=CUTOFF_LEN,
#         padding=False,
#         return_tensors=None,
#     )
#     if (
#         result["input_ids"][-1] != tokenizer.eos_token_id
#         and len(result["input_ids"]) < CUTOFF_LEN
#         and add_eos_token
#     ):
#         result["input_ids"].append(tokenizer.eos_token_id)
#         result["attention_mask"].append(1)

#     result["labels"] = result["input_ids"].copy()

#     return result

# def generate_and_tokenize_prompt(data_point):
#     full_prompt = generate_prompt(data_point)
#     tokenized_full_prompt = tokenize(full_prompt)
#     return tokenized_full_prompt

In [ ]:
# train_val = data["train"].train_test_split(
#     test_size=200, shuffle=True, seed=42
# )
# train_data = (
#     train_val["train"].map(generate_and_tokenize_prompt)
# )
# val_data = (
#     train_val["test"].map(generate_and_tokenize_prompt)
# )

Map:   0%|          | 0/10653 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# data["train"]

Dataset({
    features: ['input', 'output'],
    num_rows: 10853
})

In [ ]:
# train_val["test"][0]

{'output': '해당 기사는 낚시성기사입니다.',
 'input': "기사제목: f청신호 명동, 신혼·예비부부 위한 ‘부부, 근손실 방지 패키지’ 특강 성료 기사내용: f더 ‘잘 살고’ 싶은 청년들이 모여 주거 문화를 경험하고 주거 의제를 논의하는 커뮤니티 공간 '청신호 명동'이 지난 5월 20일부터 1달여 간 진행한 신혼부부, 예비부부 대상 온라인 특강 ‘부부, 근손실 방지 패키지’가 성황리에 종료되었다. ‘부부, 근손실 방지 패키지’는 강의명처럼, 결혼생활에서 발생하는 갈등 속에서 ‘서로를 잃지 않으면서 함께 잘 사는 가이드’를 제시하는 시리즈 특강이다.\n강의의 주제는 처음으로 함께 살게 된 신혼부부가 겪기 쉬운 일상적인 고민에 맞춰 소통, 성, 생활, 플랜테리어, 재무 총 다섯 가지 테마로 구성되었다.\n지난 5월 ‘소통’ 편으로 시작된 특강은 이달 24일 마지막 <재무>편 강좌까지 총 250여 명의 참여 속에 마무리되었다.\n특히 이번 특강은 신혼부부의 일상을 관통하는 주제선정으로 참여자의 많은 공감을 얻었다.\n누구나 있지만, 친한 이들에게도 터놓기 힘든 성생활 고민, 연애 시기와는 달리 풀어내기 어려워진 소통의 고민 등 새내기 부부의 숨은 고민을 콕콕 짚어내는 강좌에 참여자들도 높은 관심을 보였다.\n그 결과 개인차가 많이 나타날 수 있는 주제에도 불구하고 사후 설문조사에서 89% 이상의 높은 강의 만족도를 기록했다.\n가장 호응이 높았던 강의는 ‘플랜테리어’ 편이었다.\n강사는 환경 친화적 라이프스타일 브랜드 ‘더 리빙팩토리’의 대표이자, 다수의 플랜테리어 저작으로 유명한 정재경 작가로, 작은 화분 하나만으로도 집안에 새로운 활력과 분위기를 선사하는 다양한 플랜테리어 팁을 소개했다.\n특히 저렴한 비용으로도 다양하고 새로운 변화를 줄 수 있다는 점에 참가자들의 공감도가 높았고, 본 강의 시간이 끝나고도 질의가 오랫동안 계속되는 등 열띤 분위기가 이어졌다.\n정재경 작가를 비롯한 다수 유명 강사진 참여 소식도 참가자들의 관심을 끌었다.\n첫 

In [ ]:
# import transformers

# # needed for gpt-neo-x tokenizer
# tokenizer.pad_token = tokenizer.eos_token

# trainer = transformers.Trainer(
#     model=model,
#     train_dataset= train_data,
#     eval_dataset=val_data,
#     args=transformers.TrainingArguments(
#         per_device_train_batch_size=3,
#         gradient_accumulation_steps=1,
#         # warmup_steps=200,
#         max_steps=10,
#         learning_rate=5e-05,
#         fp16=True,
#         logging_steps=10,
#         output_dir="outputs",
#         optim="paged_adamw_8bit",
#         evaluation_strategy = "steps",
#         save_strategy = "steps",
#         eval_steps = 10,
#         save_steps = 10,
#         load_best_model_at_end = True,
#         report_to = "tensorboard"
#     ),
#     data_collator=transformers.DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True),
# )
# model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
# trainer.train()

Step,Training Loss,Validation Loss
10,1.693000,1.689165


TrainOutput(global_step=10, training_loss=1.6930240631103515, metrics={'train_runtime': 91.6852, 'train_samples_per_second': 0.327, 'train_steps_per_second': 0.109, 'total_flos': 2325509181112320.0, 'train_loss': 1.6930240631103515, 'epoch': 0.0})

In [ ]:
import transformers


trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    eval_dataset=data['validation'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        max_steps=500,
        learning_rate=5e-5,#0.0005
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        report_to = "tensorboard"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Step,Training Loss


KeyboardInterrupt: ignored

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from transformers import TrainingArguments, Trainer


# needed for gpt-neo-x tokenizer
##tokenizer.pad_token = tokenizer.eos_token
##opti = transformers.AdamW(model.parameters())

##scheduler =
# Define the training arguments
training_args = TrainingArguments(
    output_dif = "outputs",
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    gradient_accumulation_steps=2,
    max_steps=1000,
    learning_rate=5e-3,
    fp16=True,
    logging_steps=10,
    optim="adamw_torch",
    report_to="tensorboard",
    lr_scheduler_type="cosine",
    warmup_steps=50,  # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # Weight decay for regularization
    max_grad_norm=1.0,  # Maximum norm for the gradients (gradient clipping)
    evaluation_strategy = "steps",
    eval_steps = 100,
    load_best_model_at_end = True,
    metric_for_best_model = "eval_loss"

)

# Initialize the Trainer with the model, training arguments, training dataset, and data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data["train"],
    eval_dataset=data['validation'],
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),  # Collator for processing data
)

# Start the training process
trainer.train()


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
100,2.045400,2.034078
200,2.070900,2.028914
300,2.045800,2.026315
400,2.075000,2.024599
500,2.060500,2.020057
600,2.059900,2.017889
700,1.994000,2.016413
800,1.998900,2.014250
900,2.002600,2.013204
1000,2.069100,2.011720


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

KeyboardInterrupt: ignored

In [ ]:
!rm -r /content/outputs/runs

In [ ]:
!kill 36055

In [ ]:
!kill 36260

In [ ]:
%load_ext tensorboard
%tensorboard --logdir outputs/runs

<IPython.core.display.Javascript object>

In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=64,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [ ]:
gen('기사제목: f동료에 16번 전화 걸어 숨소리만… 고용부 직원 솜방망이 징계 기사내용: f고용노동부의 한 남성 직원이 새벽에 동료 여성 직원들에게 전화를 걸어 성적수치심을 느끼게 한 사실이 알려졌다. 그러나 직장 내 성 문제 처리에 대한 관리를 담당하는 고용부가 정작 가해 직원에게 솜방망이 징계만 내렸다. 지난 27일 SBS 보도에 따르면, 고용부 여성 직원 5명은 2019년 1월부터 이상한 전화를 받기 시작했다. 새벽에 발신 번호 표시 제한으로 걸려온 전화였는데, 이를 받아도 상대는 아무 말 없이 계속 숨소리만 냈다. 피해 직원 가운데 한 명은 5달 동안 16차례 이 같은 전화를 받았던 것으로 알려졌다. 경찰 수사 결과 이 전화는 같은 고용부 남직원 A씨가 건 것으로 확인되면서 덜미를 잡혔다. 결국 A씨는 징계위원회에 회부됐고, 고용부는 자문위원회를 열어 성희롱 여부를 판단했다. 자문위원 6명 중 5명은 “여성들이 공포감과 성적 수치심을 느끼기에 충분했다”며 이를 성희롱이라고 판단했다. 하지만 고용부는 A씨의 행동을 작은 실수로 보고 감봉 3개월의 가벼운 징계를 내렸다. 전문가들은 A씨의 행동이 고의적으로 보이는 만큼, 해임 이상의 중징계를 내렸어야 했다고 입을 모았다. 이지은 변호사는 SBS와의 인터뷰에서 “고용노동부라는 곳은 성희롱이나 성범죄에 대해서 가장 솔선을 해야 하는 부서다. 이런 식으로 본인들 스스로의 징계 의결이 솜방망이면 그것에 대해 국민들이 납득을 하겠나”라고 지적했다. 한편 A씨는 잘못을 반성하지만, 성적 의도가 없었고 징계 수위가 무겁다고 주장하고 있는 것으로 전해졌다.')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1671: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### 질문: 기사제목: f동료에 16번 전화 걸어 숨소리만… 고용부 직원 솜방망이 징계 기사내용: f고용노동부의 한 남성 직원이 새벽에 동료 여성 직원들에게 전화를 걸어 성적수치심을 느끼게 한 사실이 알려졌다. 그러나 직장 내 성 문제 처리에 대한 관리를 담당하는 고용부가 정작 가해 직원에게 솜방망이 징계만 내렸다. 지난 27일 SBS 보도에 따르면, 고용부 여성 직원 5명은 2019년 1월부터 이상한 전화를 받기 시작했다. 새벽에 발신 번호 표시 제한으로 걸려온 전화였는데, 이를 받아도 상대는 아무 말 없이 계속 숨소리만 냈다. 피해 직원 가운데 한 명은 5달 동안 16차례 이 같은 전화를 받았던 것으로 알려졌다. 경찰 수사 결과 이 전화는 같은 고용부 남직원 A씨가 건 것으로 확인되면서 덜미를 잡혔다. 결국 A씨는 징계위원회에 회부됐고, 고용부는 자문위원회를 열어 성희롱 여부를 판단했다. 자문위원 6명 중 5명은 “여성들이 공포감과 성적 수치심을 느끼기에 충분했다”며 이를 성희롱이라고 판단했다. 하지만 고용부는 A씨의 행동을 작은 실수로 보고 감봉 3개월의 가벼운 징계를 내렸다. 전문가들은 A씨의 행동이 고의적으로 보이는 만큼, 해임 이상의 중징계를 내렸어야 했다고 입을 모았다. 이지은 변호사는 SBS와의 인터뷰에서 “고용노동부라는 곳은 성희롱이나 성범죄에 대해서 가장 솔선을 해야 하는 부서다. 이런 식으로 본인들 스스로의 징계 의결이 솜방망이면 그것에 대해 국민들이 납득을 하겠나”라고 지적했다. 한편 A씨는 잘못을 반성하지만, 성적 의도가 없었고 징계 수위가 무겁다고 주장하고 있는 것으로 전해졌다.

### 답변: 해당 기사는  낚시성기사입니다. 낚시기사 유형은 사실 과대 표현형 입니다. 낚시패턴은 의도적 비방형 입니다.

### 해당 기사에서 직장동료들에게 성적수치심을 느낀다면, 전화한 동료직원은 가해직원으로서 성폭력특별법 제10조 성희롱, 성폭력


In [ ]:
gen("기사제목: f'소비자 맞춤가격' 올리브 힐 인기 기사내용: f한국토지주택공사(LH) 대전충남지역본부가 전국 최초로 분양한 '라이프 피팅 아파트' 목동 휴먼시아 '올리브 힐'이 소비자들에게 인기를 끌고 있다. LH 대전충남지역본부에 따르면 지난 6월 22일 분양에 들어간 동·층·향별 가격이 다르고 할부 및 전세 분양이 가능한 맞춤형 '라이프 피팅 아파트' 목동 '올리브 힐'이 소비자들로부터 좋은 반응을 얻고 있는 것으로 나타났다. '올리브 힐'의 분양가는 소비자들의 구미에 맞게 176가지로 다양하게 구성된 게 특징이다. 실제로, 84㎡형 로열층의 경우 분양가가 2억1500만~2억2700만원인데 반해 동이 좋지 않고 1층인 아파트는 1억 8500만원에 분양 받을 수 있다. 이에 따라 '올리브 힐'은 미분양 아파트 2개월만에 80여가구의 분양 계약이 성사되는 호성적을 올렸다. 지역부동산업계는 경기침체 등으로 매매시장에 일부 미분양 아파트가 할인가로 나오고 있는 현실을 감안할 때 이례적인 것으로 평가하고 있다. LH 대전충남지역본부도 금융권의 대출 금리인상과 부동산경기 침체 등의 악조건에도 불구하고 분양이 호조를 보이고 있는 것에 고무적이다. 지역부동산업계 한 관계자는 “소비자들이 합리적으로 아파트를 선택할 수 있도록 분양가를 다양화해 소비자들에게 인기를 끌고 있는 것 같다”고 말했다. 8개동 총 805세대로 지구주민 계약분을 제외한 635세대를 분양하는 '올리브 힐'의 분양는 3.3㎡당 630만원대 1차 분양 때 3.3㎡당 700만원대에 비하면 70만원 가량이 싸다. 10층 이하에 대해서는 전세분양이나 3년 무이자 할부분양(선납조건)이 가능한 것도 '올리브 힐'의 장점이다. 전세분양은 전세금을 내고 살다가 2년 후 분양을 원할 경우 나머지 차액을 납부하면 내집이 된다.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 기사제목: f'소비자 맞춤가격' 올리브 힐 인기 기사내용: f한국토지주택공사(LH) 대전충남지역본부가 전국 최초로 분양한 '라이프 피팅 아파트' 목동 휴먼시아 '올리브 힐'이 소비자들에게 인기를 끌고 있다. LH 대전충남지역본부에 따르면 지난 6월 22일 분양에 들어간 동·층·향별 가격이 다르고 할부 및 전세 분양이 가능한 맞춤형 '라이프 피팅 아파트' 목동 '올리브 힐'이 소비자들로부터 좋은 반응을 얻고 있는 것으로 나타났다. '올리브 힐'의 분양가는 소비자들의 구미에 맞게 176가지로 다양하게 구성된 게 특징이다. 실제로, 84㎡형 로열층의 경우 분양가가 2억1500만~2억2700만원인데 반해 동이 좋지 않고 1층인 아파트는 1억 8500만원에 분양 받을 수 있다. 이에 따라 '올리브 힐'은 미분양 아파트 2개월만에 80여가구의 분양 계약이 성사되는 호성적을 올렸다. 지역부동산업계는 경기침체 등으로 매매시장에 일부 미분양 아파트가 할인가로 나오고 있는 현실을 감안할 때 이례적인 것으로 평가하고 있다. LH 대전충남지역본부도 금융권의 대출 금리인상과 부동산경기 침체 등의 악조건에도 불구하고 분양이 호조를 보이고 있는 것에 고무적이다. 지역부동산업계 한 관계자는 “소비자들이 합리적으로 아파트를 선택할 수 있도록 분양가를 다양화해 소비자들에게 인기를 끌고 있는 것 같다”고 말했다. 8개동 총 805세대로 지구주민 계약분을 제외한 635세대를 분양하는 '올리브 힐'의 분양는 3.3㎡당 630만원대 1차 분양 때 3.3㎡당 700만원대에 비하면 70만원 가량이 싸다. 10층 이하에 대해서는 전세분양이나 3년 무이자 할부분양(선납조건)이 가능한 것도 '올리브 힐'의 장점이다. 전세분양은 전세금을 내고 살다가 2년 후 분양을 원할 경우 나머지 차액을 납부하면 내집이 된다.

### 답변: 해당 기사는  낚시성기사입니다. 낚시기사 유형은 의도적 주어 왜곡형 입니다. 낚시기사는 의도적 주어 왜곡형 입니다. 낚시기사는 의도적 주어 왜곡형 입니다. 낚시기사는 의

In [ ]:
gen("기사제목: f비축유 방출에 산유국 대응은?…\"내달 1일 OPEC·2일 OPEC+ 회의\" 기사내용: f국제유가 급등으로 산유국들의 증산 이행 여부에 관심이 쏠리는 가운데, 석유수출국기구(오펙·OPEC)는 내달 1일 관련 회의를 연다고 24일 로이터 통신이 소식통을 인용해 보도했다. 아울러 이튿날인 내달 2일에는 오펙과 러시아 등 非오펙 산유국들을 포함한 오펙 플러스(OPEC+) 회의가 열린다. 이에 앞서 오펙 플러스는 이달 29일 공동기술위원회(JTC)와 30일 장관급공동감시위원회(JMMC)도 각각 개최한다고 로이터는 전했다. 조 바이든 미국 행정부는 이날 전략비축유 5000만 배럴 방출을 내달 초 시작한다고 발표했다. 치솟는 유가를 잡기 위해 한국, 일본 등 주요 오일소비국들과의 공조 속에 이뤄지는 것이다. 다만 발표 직후 국제유가는 오히려 2~3% 급등했고, 시장의 관심은 이제 오펙 플러스의 '맞대응' 변수에 쏠리고 있다. 라쿠텐 증권 자재 애널리스트 사토루 요시다는 \"주요 오일소비국들의 공조가 오히려 오펙 플러스의 증산 속도를 늦출 것이란 불안이 커졌다\"고 시장 분위기를 전했다. 산유국들이 미국 등 오일소비국들의 이번 비축유 방출로 유가 하락 시 글로벌 오일 설비 회복에 영향을 미칠 수 있다는 점을 들어 기 약속한 증산마저 중단할 수 있다고 골드만삭스는 전망했다.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 기사제목: f비축유 방출에 산유국 대응은?…"내달 1일 OPEC·2일 OPEC+ 회의" 기사내용: f국제유가 급등으로 산유국들의 증산 이행 여부에 관심이 쏠리는 가운데, 석유수출국기구(오펙·OPEC)는 내달 1일 관련 회의를 연다고 24일 로이터 통신이 소식통을 인용해 보도했다. 아울러 이튿날인 내달 2일에는 오펙과 러시아 등 非오펙 산유국들을 포함한 오펙 플러스(OPEC+) 회의가 열린다. 이에 앞서 오펙 플러스는 이달 29일 공동기술위원회(JTC)와 30일 장관급공동감시위원회(JMMC)도 각각 개최한다고 로이터는 전했다. 조 바이든 미국 행정부는 이날 전략비축유 5000만 배럴 방출을 내달 초 시작한다고 발표했다. 치솟는 유가를 잡기 위해 한국, 일본 등 주요 오일소비국들과의 공조 속에 이뤄지는 것이다. 다만 발표 직후 국제유가는 오히려 2~3% 급등했고, 시장의 관심은 이제 오펙 플러스의 '맞대응' 변수에 쏠리고 있다. 라쿠텐 증권 자재 애널리스트 사토루 요시다는 "주요 오일소비국들의 공조가 오히려 오펙 플러스의 증산 속도를 늦출 것이란 불안이 커졌다"고 시장 분위기를 전했다. 산유국들이 미국 등 오일소비국들의 이번 비축유 방출로 유가 하락 시 글로벌 오일 설비 회복에 영향을 미칠 수 있다는 점을 들어 기 약속한 증산마저 중단할 수 있다고 골드만삭스는 전망했다.

### 답변: 해당 기사는  낚시성기사입니다. 낚시기사 유형은 의문 유발형(부호) 입니다. 낚시기사 유형은 의문 유발형(부호) 입니다. 낚시기사의 유형을 보면 의도적 노출/부호 사용형과 의도적 노출/부호 사용형으로 구분할 수 있


In [ ]:
gen("기사제목: f현대건설 부천 공사현장 안전사고...사상자는 없어 기사내용: f18일 오전 경기 부천시 범박동 한 현대건설 공사현장에서 28층에 설치된 타워크레인 구조물 일부가 끊어져 추락하는 사고가 발생했다. 인명피해는 발생하지 않은 것으로 나타났다. 이날 부천시 관계자는 데일리임팩트와 통화에서 \"구조물을 연결하는 와이어가 끊어져 발생한 사고\"라며 \"사망자나 다친 사람은 없고 정확한 사고원인은 시에서 조사 중\"이라고 말했다. 현대건설 안전팀은 정확한 사고경위를 조사해 조만간 시에 제출할 예정이다. 국토교통부에 따르면 현대건설에는 지난 2021년 미집계 된 4분기를 제외하고 매분기 안전사고가 발생했다. 이에 따라 현재 현대건설은 현장 안전 관리를 위해 실시간 통합 관리 모니터링 시스템을 개발·적용 중이다.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 기사제목: f현대건설 부천 공사현장 안전사고...사상자는 없어 기사내용: f18일 오전 경기 부천시 범박동 한 현대건설 공사현장에서 28층에 설치된 타워크레인 구조물 일부가 끊어져 추락하는 사고가 발생했다. 인명피해는 발생하지 않은 것으로 나타났다. 이날 부천시 관계자는 데일리임팩트와 통화에서 "구조물을 연결하는 와이어가 끊어져 발생한 사고"라며 "사망자나 다친 사람은 없고 정확한 사고원인은 시에서 조사 중"이라고 말했다. 현대건설 안전팀은 정확한 사고경위를 조사해 조만간 시에 제출할 예정이다. 국토교통부에 따르면 현대건설에는 지난 2021년 미집계 된 4분기를 제외하고 매분기 안전사고가 발생했다. 이에 따라 현재 현대건설은 현장 안전 관리를 위해 실시간 통합 관리 모니터링 시스템을 개발·적용 중이다.

### 답변: 해당 기사는  낚시성기사입니다. 낚시기사 유형은 속어/줄임말 사용형 입니다. 낚시기사 유형 보기 상품 판매정보 상품 판매글·사진 상품 판매정보 상품 정가1,145,000상품 가격1,080,000 상품명현대건설명정주계약자


In [ ]:
gen("f허 찌르는 농담이 분위기 띄우는 진짜 이유 기사내용: f긴장된 분위기 속에서 연설자가 청중에게 의외의 농담을 던지면 분위기가 좋아진다. 미국 건강포털 웹엠디 등이 소개한 연구 결과에 따르면 농담이 이러한 효과를 발휘하는 것은 듣는 사람의 정신을 잠깐 빼앗기 때문이라고 한다. 예상을 깨는 농담일수록 그 뜻을 알아채려면 머리를 써야 하고 그렇게 하는 순간에 그 이전의 긴장된 기분과 상태를 자연스럽게 잊어버리게 된다는 것이다. 네덜란드 라드바우드 대학의 마델린 스트릭 연구원은 또한 이러한 효과를 긍정적 단어가 불러오지 않는다는 것을 실험을 통해 증명했다. 그는 평균 나이 21.5세인 대학생 90명에게 긍정,부정,중성적인 세 가지 사진을 보여주고 느낌을 물었다. 실험 참가자들이 대답 하기 전 농담 또는 긍정적인 느낌을 주는 말을 하고 그 효과를 알아보았다. 그러자 부정적인 사진을 보고 농담을 들었을 때 사진에 대해 부정적으로 느끼는 정도가 가장 크게 줄어드는 것으로 나타났다. 단순히 긍정적인 말로는 이런 효과를 거둘 수 없었다. 긍정적, 중성적 사진을 봤을 때는 큰 차이가 없었다. 연구진은 이런 결과에 대해 “예측할 수 있는 뻔한 농담은 별로 효과가 없으며, 허를 찌르는 참신한 농담이라야 효과가 커진다”고 설명했다. 조인스닷컴 헬스케어 편집팀")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: f허 찌르는 농담이 분위기 띄우는 진짜 이유 기사내용: f긴장된 분위기 속에서 연설자가 청중에게 의외의 농담을 던지면 분위기가 좋아진다. 미국 건강포털 웹엠디 등이 소개한 연구 결과에 따르면 농담이 이러한 효과를 발휘하는 것은 듣는 사람의 정신을 잠깐 빼앗기 때문이라고 한다. 예상을 깨는 농담일수록 그 뜻을 알아채려면 머리를 써야 하고 그렇게 하는 순간에 그 이전의 긴장된 기분과 상태를 자연스럽게 잊어버리게 된다는 것이다. 네덜란드 라드바우드 대학의 마델린 스트릭 연구원은 또한 이러한 효과를 긍정적 단어가 불러오지 않는다는 것을 실험을 통해 증명했다. 그는 평균 나이 21.5세인 대학생 90명에게 긍정,부정,중성적인 세 가지 사진을 보여주고 느낌을 물었다. 실험 참가자들이 대답 하기 전 농담 또는 긍정적인 느낌을 주는 말을 하고 그 효과를 알아보았다. 그러자 부정적인 사진을 보고 농담을 들었을 때 사진에 대해 부정적으로 느끼는 정도가 가장 크게 줄어드는 것으로 나타났다. 단순히 긍정적인 말로는 이런 효과를 거둘 수 없었다. 긍정적, 중성적 사진을 봤을 때는 큰 차이가 없었다. 연구진은 이런 결과에 대해 “예측할 수 있는 뻔한 농담은 별로 효과가 없으며, 허를 찌르는 참신한 농담이라야 효과가 커진다”고 설명했다. 조인스닷컴 헬스케어 편집팀

### 답변: 해당 기사는  낚시성기사입니다. 낚시기사 유형은 의도적 주어 왜곡형 입니다. 낚시기사 유형은 의문 유발형 입니다. 의문 유발형은 사실과 다르게 표현 하거나 왜곡되게 표현 합니다. 의문 유발형은 의도적 주어 왜곡형 입니다. 의도적 주어 왜곡형


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

model.push_to_hub("re2panda/polyglot_1.3B_click_bate_test_dataset_split_1103", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:853: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


LocalTokenNotFoundError: ignored

In [ ]:
model.push_to_hub("re2panda/polyglot_1.3B_click_bate_test_dataset_split_1103", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:853: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/re2panda/polyglot_1.3B_click_bate_test_dataset_split_1103/commit/bdd72acf89339e50e6721cf1cd08d4f7283b4a1b', commit_message='Upload model', commit_description='', oid='bdd72acf89339e50e6721cf1cd08d4f7283b4a1b', pr_url=None, pr_revision=None, pr_num=None)